task)

1. CSP, FFT 이용하여 데이터 전처리
2. 머신러닝 기법(로지스틱)으로 데이터 분류
3. 전처리 데이터 fc layer에 넣어서 결과 확인하기(은닉층 3개 정도)

In [1]:
#moabb 데이터셋 불러오기

from moabb.datasets import BNCI2014_008
from moabb.paradigms import P300

# 데이터셋과 패러다임 설정
dataset = BNCI2014_008()
paradigm = P300()  # MotorImagery() 대신 P300() 사용

# 데이터 불러오기
X, labels, metadata = paradigm.get_data(dataset)

# 데이터 확인
print(X.shape)
print(labels[:5])
print(metadata.head())

<frozen importlib._bootstrap>:219: FutureWarning: mne.io.pick.pick_types is deprecated will be removed in 1.6, use documented public API instead. If no appropriate public API exists, please open an issue on GitHub.
<frozen importlib._bootstrap>:219: FutureWarning: mne.io.pick.pick_channels_regexp is deprecated will be removed in 1.6, use documented public API instead. If no appropriate public API exists, please open an issue on GitHub.
<frozen importlib._bootstrap>:219: FutureWarning: mne.io.pick.channel_type is deprecated will be removed in 1.6, use documented public API instead. If no appropriate public API exists, please open an issue on GitHub.
C:\Users\sangm\anaconda3\envs\moabb\lib\site-packages\moabb\pipelines\__init__.py:26: ModuleNotFoundError: Tensorflow is not installed. You won't be able to use these MOABB pipelines if you attempt to do so.
  warn(
C:\Users\sangm\anaconda3\envs\moabb\lib\site-packages\sklearn\preprocessing\_function_transformer.py:312: FutureWarning: The cu

(33600, 8, 257)
['NonTarget' 'NonTarget' 'NonTarget' 'NonTarget' 'NonTarget']
   subject session run
0        1       0   0
1        1       0   0
2        1       0   0
3        1       0   0
4        1       0   0


[CSP와 FFT를 이용하여 데이터 전처리]

CSP 4개 특성, FFT 8개 특성 추출하여 총 12개 특성 사용

In [2]:
#csp와 fft를 적용하여 데이터 전처리

import numpy as np
from mne.decoding import CSP
from scipy.fft import fft
from sklearn.preprocessing import StandardScaler

# ✅ 원본 데이터 (X: (samples, channels, timepoints))
print("Original shape:", X.shape)  # (4200, 8, 256)

# 1️⃣ CSP 적용 (n_components=4)
csp = CSP(n_components=4, reg=None, log=True, norm_trace=False)
X_csp = csp.fit_transform(X, labels)
print("After CSP:", X_csp.shape)  # (4200, 4)

# 2️⃣ FFT 적용 (주파수 변환 후 평균)
def compute_fft(X):
    X_fft = np.abs(fft(X, axis=2))  # FFT 적용 후 절댓값
    return X_fft.mean(axis=2)  # 시간축 평균 계산

X_fft = compute_fft(X)
print("After FFT:", X_fft.shape)  # (4200, 8)

# 3️⃣ 최종 데이터 (CSP + FFT 특징 결합)
X_features = np.hstack((X_csp, X_fft))
scaler = StandardScaler()
X_features = scaler.fit_transform(X_features)

print("Final processed shape:", X_features.shape)  # (4200, 12)


Original shape: (33600, 8, 257)
Computing rank from data with rank=None
    Using tolerance 1e+02 (2.2e-16 eps * 8 dim * 5.8e+16  max singular value)
    Estimated rank (mag): 8
    MAG: rank 8 computed from 8 data channels with 0 projectors
Reducing data rank from 8 -> 8
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Using tolerance 47 (2.2e-16 eps * 8 dim * 2.6e+16  max singular value)
    Estimated rank (mag): 8
    MAG: rank 8 computed from 8 data channels with 0 projectors
Reducing data rank from 8 -> 8
Estimating covariance using EMPIRICAL
Done.
After CSP: (33600, 4)
After FFT: (33600, 8)
Final processed shape: (33600, 12)


X_features : (4200, 12) → 분류 모델의 입력 데이터

-> (샘플 수, CSP 특징 개수 + FFT 특징 개수)


labels : (4200,) → 레이블 (0 = NT, 1 = T)

In [ ]:
[여러가지 이진분류 모델 사용하여 분류]

전처리된 EEG 데이터가 어느 타깃 T(1), NT(0)에 속하는지 분류

In [3]:
#여러가지 이진분류 모델로 주어진 EEG 신호가 NT인지 T인지 이진분류

import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

# ✅ 데이터셋 (CSP + FFT 특징)
X = X_features  # 전처리된 특징 데이터 (CSP + FFT)
labels = np.array(labels)  # 이진 레이블 (0: NT, 1: T)

# ✅ Train-Test 데이터 분할
X_train, X_test, labels_train, labels_test = train_test_split(X, labels, test_size=0.2, random_state=42)

# ✅ 여러 분류 모델 정의
models = {
    "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42),
    "Logistic Regression": LogisticRegression(),
    "LDA": LDA(),
    "KNN": KNeighborsClassifier(n_neighbors=5),
    "SVM": SVC(kernel='rbf', C=1, probability=True)
}

# ✅ 각 모델 학습 및 평가
for name, model in models.items():
    print(f"Training {name}...")  # 훈련 시작 메시지 출력
    model.fit(X_train, labels_train)  # 모델 학습
    labels_pred = model.predict(X_test)  # 예측 수행
    accuracy = accuracy_score(labels_test, labels_pred)  # 정확도 계산
    print(f"{name} Accuracy: {accuracy:.4f}")  # 정확도 출력

Training Random Forest...
Random Forest Accuracy: 0.8363
Training Logistic Regression...
Logistic Regression Accuracy: 0.8366
Training LDA...
LDA Accuracy: 0.8366
Training KNN...
KNN Accuracy: 0.8107
Training SVM...
SVM Accuracy: 0.8366


#SVM

support vector machine

support : 데이터의 경계선

vector : 직선

'데이터 간의 경계들과 최대한 멀리 떨어져있는 선'을 찾는 것(각 데이터 경계와 선 사이의 여백이 최대가 되도록 하는 선을 찾는 것)

다중분류 문제에서도 사용 가능

[FC layer 사용하여 분류]

step2에서는 각종 머신러닝 이진분류 모델을 사용했고,

이제는 여러 은닉층을 두는 '딥러닝' 으로 분류 수행

3개의 은닉층으로 설계


✅ Fully Connected (FC) 신경망 구조
✅ 은닉층 3개 (ReLU 활성화 함수 사용)
✅ 출력층 (이진 분류 → 시그모이드 활성화 함수 사용)
✅ Adam 옵티마이저 + Binary Crossentropy 손실 함수 사용
✅ Train/Test 분할 후 학습 및 평가 수행

In [9]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# ✅ 데이터 준비 (CSP + FFT 특징)
X = X_features  # 전처리된 특징 데이터

# ✅ 문자열 라벨을 0과 1로 변환 (Target → 1, NonTarget → 0)
y = np.array([1 if label == "Target" else 0 for label in labels], dtype=np.float32)

# ✅ 변환 확인
print("Unique labels after conversion:", np.unique(y))  # [0. 1.]이 나와야 정상

# ✅ 데이터 정규화 (표준화)
scaler = StandardScaler()
X= scaler.fit_transform(X)

# ✅ Train-Test 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# ✅ FC Layer 기반 모델 생성
model = keras.Sequential([
    keras.layers.Dense(512, activation='relu', input_shape=(X.shape[1],)),  # 첫 번째 은닉층
    keras.layers.Dense(256, activation='relu'),  # 두 번째 은닉층
    keras.layers.Dense(128, activation='relu'),  # 세 번째 은닉층
    keras.layers.Dense(1, activation='sigmoid')  # 출력층 (이진 분류 → sigmoid)
])

# ✅ 모델 컴파일 (Binary Crossentropy, Adam 옵티마이저)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# ✅ 모델 학습
history = model.fit(X_train, y_train, epochs=50, batch_size=16, validation_data=(X_test, y_test))

# ✅ 최종 정확도 평가
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_acc:.4f}")

#학습률 확인
learning_rate = model.optimizer.learning_rate.numpy()
print(f"Current Learning Rate: {learning_rate}")


Unique labels after conversion: [0. 1.]
Epoch 1/50
1680/1680 [==============================] - 5s 3ms/step - loss: 0.4598 - accuracy: 0.8324 - val_loss: 0.4454 - val_accuracy: 0.8366
Epoch 2/50
1680/1680 [==============================] - 4s 2ms/step - loss: 0.4519 - accuracy: 0.8325 - val_loss: 0.4439 - val_accuracy: 0.8366
Epoch 3/50
1680/1680 [==============================] - 4s 2ms/step - loss: 0.4509 - accuracy: 0.8325 - val_loss: 0.4450 - val_accuracy: 0.8366
Epoch 4/50
1680/1680 [==============================] - 4s 3ms/step - loss: 0.4509 - accuracy: 0.8325 - val_loss: 0.4441 - val_accuracy: 0.8366
Epoch 5/50
1680/1680 [==============================] - 4s 3ms/step - loss: 0.4502 - accuracy: 0.8325 - val_loss: 0.4443 - val_accuracy: 0.8366
Epoch 6/50
1680/1680 [==============================] - 4s 3ms/step - loss: 0.4501 - accuracy: 0.8325 - val_loss: 0.4429 - val_accuracy: 0.8366
Epoch 7/50
1680/1680 [==============================] - 4s 2ms/step - loss: 0.4500 - accuracy: 0

특성 12개로 학습한 머신러닝 모델의 최고 성능 : 0.8366
특성 12개로 학습한 딥러닝 모델의 최고 성능 : 0.8366



원본 데이터 shape (33600, 8, 257)
-> 모델이 학습할 수 있는 특성의 개수 8x257개(feature 수)

근데 이제

csp, fft를 이용해서 특성을 4 + 8 = 12개로 줄여버림

사실상 특성이 2056개에서 12개로 줄어든 것


여러 이진분류 모델을 사용한 step2에서는 머신러닝을 사용했기 때문에 12개의 특성이 부족하지 않을 수 있지만,

step3에서 fc layer을 이용한 딥러닝 모델을 학습할 경우, 12개의 특성은 너무너무 작은 개수임

따라서, 이제부터는 csp, fft를 이용하여 특성의 개수를 제한하지 않고, 
단순히 원본 데이터를 flatten하여 fc layer에 전달하는 방식으로 충분한 특성 수를 보장하여 딥러닝 모델을 학습시킬 것임


In [13]:
#CSP, FFT 를 사용하지 않고 단순 flatten한 데이터 사용

#moabb 데이터셋 불러오기

from moabb.datasets import BNCI2014_008
from moabb.paradigms import P300

# 데이터셋과 패러다임 설정
dataset = BNCI2014_008()
paradigm = P300()  # MotorImagery() 대신 P300() 사용

# 데이터 불러오기
origin, labels, metadata = paradigm.get_data(dataset)

# 데이터 확인
print(origin.shape)
print(labels[:5])
print(metadata.head())

C:\Users\sangm\anaconda3\envs\moabb\lib\site-packages\sklearn\preprocessing\_function_transformer.py:312: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  return func(X, **(kw_args if kw_args else {}))
C:\Users\sangm\anaconda3\envs\moabb\lib\site-packages\sklearn\preprocessing\_function_transformer.py:312: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  return func(X, **(kw_args if kw_args else {}))
C:\Users\sangm\anaconda3\envs\moabb\lib\site-packages\sklearn\preprocessing\_function_transformer.py:312: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  return func(X, **(kw_args if kw_args else {}))
C:\Users\sangm\anaconda3\envs\moabb\lib\site-packages\sklearn\preprocessing\_function_transformer.py:312: FutureWarning: The

(33600, 8, 257)
['NonTarget' 'NonTarget' 'NonTarget' 'NonTarget' 'NonTarget']
   subject session run
0        1       0   0
1        1       0   0
2        1       0   0
3        1       0   0
4        1       0   0


C:\Users\sangm\anaconda3\envs\moabb\lib\site-packages\sklearn\preprocessing\_function_transformer.py:312: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  return func(X, **(kw_args if kw_args else {}))


In [14]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# ✅ 원본 EEG 데이터 사용 (CSP & FFT 없이)
X = np.array(origin)  # 원본 EEG 데이터 (shape: samples, channels, timepoints)
y = np.array([1 if label == "Target" else 0 for label in labels], dtype=np.float32)  # 이진 라벨 변환

# ✅ 데이터 확인
print(f"Original X shape: {X.shape}")  # (샘플 수, 채널 수, 시간 포인트 수)
print(f"Unique labels: {np.unique(y)}")  # [0. 1.]

# ✅ Flatten 적용 (2D 벡터로 변환)
X = X.reshape(X.shape[0], -1)  # (샘플 수, 채널 수 * 시간 포인트 수)
print(f"Flattened X shape: {X.shape}")  # (샘플 수, 전체 특성 수)

# ✅ 데이터 정규화 (표준화)
scaler = StandardScaler()
X = scaler.fit_transform(X)

# ✅ Train-Test 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# ✅ FC Layer 기반 모델 생성
# 결합되는 채널 수를 임의로 늘림(모델을 더 복잡하게 만듦)
model = keras.Sequential([
    keras.layers.Dense(2048, activation='relu', input_shape=(X.shape[1],)),  # 첫 번째 은닉층
    keras.layers.BatchNormalization(),  # 배치 정규화 추가
    keras.layers.Dropout(0.3),  # 과적합 방지
    
    keras.layers.Dense(1024, activation='relu'),  # 두 번째 은닉층
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.3),
    
    keras.layers.Dense(512, activation='relu'),  # 세 번째 은닉층
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.3),
    
    keras.layers.Dense(1, activation='sigmoid')  # 출력층 (이진 분류 → sigmoid)
])

# ✅ 모델 컴파일 (Binary Crossentropy, Adam 옵티마이저)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# ✅ 모델 학습
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))

# ✅ 최종 정확도 평가
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_acc:.4f}")

#학습률 확인
learning_rate = model.optimizer.learning_rate.numpy()
print(f"Current Learning Rate: {learning_rate}")

Original X shape: (33600, 8, 257)
Unique labels: [0. 1.]
Flattened X shape: (33600, 2056)
Epoch 1/50
840/840 [==============================] - 34s 39ms/step - loss: 0.5129 - accuracy: 0.7918 - val_loss: 0.4151 - val_accuracy: 0.8269
Epoch 2/50
840/840 [==============================] - 34s 40ms/step - loss: 0.3991 - accuracy: 0.8376 - val_loss: 0.3831 - val_accuracy: 0.8423
Epoch 3/50
840/840 [==============================] - 45s 53ms/step - loss: 0.3752 - accuracy: 0.8455 - val_loss: 0.4003 - val_accuracy: 0.8345
Epoch 4/50
840/840 [==============================] - 58s 68ms/step - loss: 0.3607 - accuracy: 0.8496 - val_loss: 0.3848 - val_accuracy: 0.8418
Epoch 5/50
840/840 [==============================] - 69s 82ms/step - loss: 0.3493 - accuracy: 0.8531 - val_loss: 0.3829 - val_accuracy: 0.8454
Epoch 6/50
840/840 [==============================] - 61s 73ms/step - loss: 0.3417 - accuracy: 0.8570 - val_loss: 0.3797 - val_accuracy: 0.8452
Epoch 7/50
840/840 [==========================

(훈련/검증 세트) 2개로 나눔


에포크 7에서 검증 세트 점수 최대치(85.21%)를 찍음


에포크8 이후부터 검증세트 점수가 떨어지고, 훈련세트 점수는 급격히 오름

-> 이를 통해 에포크 7에서 최고 성능 85.21%를 찍고, 이후 에포크부터 점점 모델이 과대적합 됨을 알 수 있음

---------------------------------------------------------------------------------------------------------------------------------

조정 가능한 하이퍼파라미터)

- dropout
- 뉴런수
- 학습률(잘 학습되는 것을 보니, 조정 안하는게 좋을듯)